# Classification

### Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sklearn

%matplotlib inline

In [ ]:
from random import random
from sklearn import datasets as skdataset
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.preprocessing import StandardScaler

In [ ]:
RAND_ST = 42

### Utility functions

##### Data loading

In [ ]:
def load_iris_dataset(num_classes=2):
    iris = skdataset.load_iris()
    X = iris.data
    Y = iris.target
    
    # reduce number of classes
    idx = Y < num_classes
    X = X[idx]
    Y = Y[idx]
    
    return X, Y

In [ ]:
def load_circle_dataset(num_samples=100):
    X, Y = skdataset.make_circles(num_samples, factor=.1, noise=.1)
    return X, Y

In [ ]:
def load_digit_dataset(num_classes=2):
    X, Y = skdataset.load_digits(n_class=num_classes, return_X_y=True)
    return X, Y

In [ ]:
def load_wine_dataset(num_classes=2):
    X, Y = skdataset.load_wine(return_X_y=True)
    
    # reduce number of classes
    idx = Y < num_classes
    X = X[idx]
    Y = Y[idx]
    
    return X, Y

In [ ]:
def load_mnist_dataset(num_classes=2):
    train_samples = 5000

    X, Y = skdataset.fetch_openml('mnist_784', version=1, return_X_y=True)
    Y = Y.astype(np.int64)
    
    # reduce number of classes
    idx = Y < num_classes
    X = X[idx]
    Y = Y[idx]

    return X, Y


def load_mnist_dataset_onevsall(class_id=7):
    train_samples = 5000

    X, Y = skdataset.fetch_openml('mnist_784', version=1, return_X_y=True)
    Y = Y.astype(np.int64)
    
    # reduce number of classes
    idx = Y == class_id
    Y[~idx] = 0
    Y[idx]  = 1

    return X, Y

##### Plot

In [ ]:
def plot_dataset(X, Y, dims=[0,1]):
    
    # slice dataset
    X_reduced = X[:, dims]
    
    # plot axis limits
    #x_min, x_max = X_reduced[:, 0].min() - .5, X_reduced[:, 0].max() + .5
    #y_min, y_max = X_reduced[:, 1].min() - .5, X_reduced[:, 1].max() + .5

    plt.figure(figsize=(8, 6))
    plt.clf()

    # Plot the training points
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=Y, cmap=plt.cm.Set1, edgecolor='k')
    plt.xlabel('Feat 1')
    plt.ylabel('Feat 2')
    
    plt.show()


def plot_pca(X, Y):
    # Plot the first three PCA dimensions
    fig = plt.figure(figsize=(8, 6))
    ax = Axes3D(fig, elev=-150, azim=110)
    
    X_pca = PCA(n_components=3).fit_transform(X)
    ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=Y,
               cmap=plt.cm.Set1, edgecolor='k', s=40)
    
    ax.set_title("First three PCA directions")
    ax.set_xlabel("1st eigenvector")
    ax.w_xaxis.set_ticklabels([])
    ax.set_ylabel("2nd eigenvector")
    ax.w_yaxis.set_ticklabels([])
    ax.set_zlabel("3rd eigenvector")
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [ ]:
def plot_loss_curve(logs):
    
    fig = plt.figure(figsize=(8, 6))
    plt.plot(logs)
    plt.show()

In [ ]:
def plot_linear_classifier(X, Y, W, dims=[0,1]):
    
    b = W[-1]
    W = W[:-1]
    w = W[dims]
        
    # slice dataset
    X_reduced = X[:, dims]
    
    fig = plt.figure(figsize=(8, 6))
    plt.clf()
    
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=Y, cmap=plt.cm.Set1, edgecolor='k')
    plt.xlabel('Feat 1')
    plt.ylabel('Feat 2')
    
    m = -w[0] / w[1]
    q = -(b-0.5) / w[1]

    xx = np.linspace(X_reduced[:, 0].min(), X_reduced[:, 0].max())
    yy = m*xx + q
    
    plt.plot(xx, yy, 'k-')
    plt.show()
    

In [ ]:
def plot_digit(sample):
    fig = plt.figure(figsize=(8,6))
    plt.gray() 
    plt.matshow(sample.reshape(8,8))
    plt.show()

def plot_mnist_digit(sample):
    fig = plt.figure(figsize=(8,6))
    plt.gray() 
    plt.matshow(sample.reshape(28,28))
    plt.show()

###### Performance Meausre

In [ ]:
def accuracy(Y_test, y_pred):
    
    accuracy = np.sum(Y_test == y_pred)
    accuracy /= Y_test.shape[0]
    
    return accuracy

def precision(Y_test, y_pred):
    
    mask = (Y_test == 1)
    
    precision = np.sum(Y_test[mask] == y_pred[mask])
    precision /= np.sum(mask.astype(np.float64))
    
    return precision


In [ ]:
def true_positive(Y_test, y_pred):
    mask = (Y_test == 1)
    
    tp = (Y_test[mask] == y_pred[mask]).sum()
    
    return tp.item()

def true_negative(Y_test, y_pred):
    mask = (Y_test == 0) | (Y_test == -1)
    
    tn = (Y_test[mask] == y_pred[mask]).sum()
    
    return tn.item()

def false_negative(Y_test, y_pred):
    mask = (y_pred == 0) | (y_pred == -1)
    
    tn = (Y_test[mask] != y_pred[mask]).sum()
    
    return tn.item()

def false_positive(Y_test, y_pred):
    mask = (y_pred == 1)
    
    tn = (Y_test[mask] != y_pred[mask]).sum()
    
    return tn.item()

def plot_confusion_matrix(Y_test, y_pred):
    
    tp = true_positive(Y_test, y_pred)
    tn = true_negative(Y_test, y_pred)
    fp = false_positive(Y_test, y_pred)
    fn = false_negative(Y_test, y_pred)
    
    cf = np.array([[tn, fn], [fp, tp]])
    
    fig, ax = plt.subplots()

    ax.matshow(cf, cmap=plt.cm.Blues)

    for i in range(2):
        for j in range(2):
            c = cf[j,i]
            ax.text(i, j, str(c), va='center', ha='center')

    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.show()


## Linear Classifier

##### Load data

In [ ]:
X, Y = load_iris_dataset()

In [ ]:
plot_dataset(X, Y, dims=[0,2])

In [ ]:
X = X[:, [0,2]]

In [ ]:
plot_dataset(X, Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
plot_dataset(X_train, Y_train, dims=[0,1])

#### Compute Model Parameters

#### Weights of a linear classifiers can be computed in closed form.

Please find the optimal weights for a linear classifier using LS.

Hint:

\begin{align}
W_{opt} = (X^T X)^{-1} X^T y
\end{align}


In [ ]:
# Students please fill here

#### Check your solution

In [ ]:
plot_linear_classifier(X_train, Y_train, W_opt)

##### Check results using confusion matrix

In [ ]:
y_pred_train = (np.matmul(x_train, W_opt) > 0.5).astype(np.int64)
y_pred_test  = (np.matmul(x_test, W_opt) > 0.5).astype(np.int64)

In [ ]:
plot_confusion_matrix(Y_train, y_pred_train)

In [ ]:
plot_confusion_matrix(Y_test, y_pred_test)

### Fail case

Linear classifiers does not work in simple datasets as the one shown below

In [ ]:
X, Y = load_circle_dataset(num_samples=1000)

In [ ]:
plot_dataset(X, Y)

###### Check that a linear classifier will actually fail

Please compute model weights with LS over a training set, then test your solution on the test set.

To check your results use the confusion matrix as above

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

#### Weights of a linear classifiers can be computed in closed form.

Please find the optimal weights for a linear classifier using LS.

Hint:

\begin{align}
W_{opt} = (X^T X)^{-1} X^T y
\end{align}


In [ ]:
# Students please fill here

## Logistic Classifier

##### Training procedure

In [ ]:
def train_logistic_classifier(X, Y, W, loss_function, gradient, step_size, max_it):
    
    checkpoint_step = int(max_it / 10)
    
    best_loss = float('+inf')
    best_w = None
    
    history = []
    for it in range(max_it):
        
        z = np.dot(X, W)
        prediction = sigmoid(z)
            
        loss = loss_function(prediction, Y)
        
        if loss < best_loss:
            best_w = W
            
        J = gradient(prediction, X, Y)
        W = W - step_size * J
        
        history.append(loss)
            
        if (it + 1) % checkpoint_step == 0:
            print('[{:05}] current loss: {}'.format(it+1, loss))
    
    return history, best_w

#### Cross entropy loss

Please implement the cross entropy loss.

Hint:

\begin{equation}
L = - \sum y * log(z) - \sum (1-y) * log(1-z)
\end{equation}

Gradient:
\begin{equation}
    \frac{\partial L}{\partial z} = \frac{z-y}{y(1-y)}
\end{equation}

In [ ]:
def loss_function(preds, targets):

    cost = float('+inf')
    # Students please fill here
    
    

    return cost

In [ ]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

#### Gradient

Please implement the gradient function below.

Keep in mind the gradient should take into account both the loss function and the model.


In [ ]:
def gradient(preds, X, Y):
    # Students please fill here
    

    return J

##### Load data

In [ ]:
X, Y = load_digit_dataset(num_classes=2)

In [ ]:
X.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

##### Data normalization

In [ ]:
X_train /= 16
X_test /= 16

In [ ]:
plot_digit(X[0])

In [ ]:
plot_digit(X_train[10])

##### Definition model parameters

In [ ]:
x_train = np.hstack((X_train, np.ones((X_train.shape[0],1))))
x_test = np.hstack((X_test, np.ones((X_test.shape[0],1))))

W = np.random.uniform(-1, 1, size=(x_train.shape[1]))

##### Train

In [ ]:
step_size = 0.001
max_it = 10000

In [ ]:
history, W_opt = train_logistic_classifier(x_train/16, Y_train, W, loss_function, gradient, step_size, max_it)

In [ ]:
plot_loss_curve(history)

###### Make prediction

Please compute the binary classification for both the training and test samples.

**Hint**: binary classification implies class are integer number.

In [ ]:
# Students please fill here
y_pred_train = 
y_pred_test  = 

In [ ]:
plot_confusion_matrix(Y_train, y_pred_train)

In [ ]:
plot_confusion_matrix(Y_test, y_pred_test)

## Hinge Loss

##### Training procedure

In [ ]:
def train_hinge_classifier(X, Y, W, loss_function, gradient, step_size, max_it):
    
    checkpoint_step = int(max_it / 10)
    
    one = np.ones(Y.shape[0])
    best_w = np.zeros(W.shape)
    best_loss = float('+inf')
    
    history = []
    for it in range(max_it):
            
        z = np.dot(X, W)

        loss = loss_function(z, Y)

        if loss < best_loss:
            best_w = np.copy(W)

        J_w = gradient(z, W, X, Y)

        W = W - step_size * J_w

        history.append(loss)

        if (it + 1) % checkpoint_step == 0:
            print('[{:05}] current loss: {}'.format(it, loss))
    
    return history, best_w

In [ ]:
def sgn_predict(W, X):
    z = np.dot(X, W)
    out = np.sign(z)
    return out

###### Gradient function

Please implement the gradient of the hinge loss w/ linear model

In [ ]:
def gradient(preds, W, X, Y):
    J = None
    # Students please fill here
    
    
    return J

##### Loss function

Please implement hinge loss given the dot product:

\begin{equation}
z = wx
\end{equation}

In [ ]:
def hinge_loss(z, Y):
    loss = None
    # Students please fill here
    
    return loss

##### Dataset loading

In [ ]:
X, Y = load_iris_dataset(num_classes=2)

Set negative class to -1 instead of 0

In [ ]:
Y[Y == 0] = -1

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

##### Definition model parameters

In [ ]:
x_train = np.hstack((X_train, np.ones((X_train.shape[0], 1))))
x_test = np.hstack((X_test, np.ones((X_test.shape[0], 1))))

W = np.zeros(x_train.shape[1])

##### Train

In [ ]:
C = 1.0
step_size = 0.01
max_it = 1000

In [ ]:
history, W_opt = train_hinge_classifier(x_train, y_train, W, hinge_loss, gradient, step_size, max_it)

In [ ]:
plot_loss_curve(history)

#### Performance check

Dataset is linearly separable so we expect this to predict correctly all the time.

Please check with a confusion matrix the above claim.

**Hint**: binary classification implies class are integer number.

In [ ]:
# Students please fill here
y_pred_train = 
y_pred_test  = 

In [ ]:
plot_confusion_matrix(y_train, y_pred_train)

In [ ]:
plot_confusion_matrix(y_test, y_pred_test)

## SVM

In [ ]:
X, Y = load_mnist_dataset(num_classes=2)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
# normalize data
X_train /= 255.0
X_test /= 255.0

In [ ]:
plot_mnist_digit(X_train[0])

In [ ]:
linsvm = svm.SVC(C=0.1, kernel='linear')

In [ ]:
linsvm.fit(X_train, Y_train)

In [ ]:
preds = linsvm.predict(X_test)
plot_confusion_matrix(Y_test, preds)

In [ ]:
preds = linsvm.predict(X_train)
plot_confusion_matrix(Y_train, preds)

#### SVM & circles

In [ ]:
X, Y = load_circle_dataset(num_samples=1000)

In [ ]:
plot_dataset(X, Y)

In [ ]:
Y[Y==0]=-1

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
linsvm = svm.SVC(C=1.0, kernel='linear')

In [ ]:
linsvm.fit(X_train, Y_train)

In [ ]:
preds = linsvm.predict(X_test)
plot_confusion_matrix(Y_test, preds)

In [ ]:
preds = linsvm.predict(X_train)
plot_confusion_matrix(Y_train, preds)

## Kernel SVM

##### Dataset loading

In [ ]:
X, Y = load_circle_dataset(num_samples=1000)

In [ ]:
plot_dataset(X, Y)

In [ ]:
Y[Y==0]=-1

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
polysvm = svm.SVC(kernel='poly', degree=2)

In [ ]:
polysvm.fit(X_train, Y_train)

In [ ]:
preds = polysvm.predict(X_test)
plot_confusion_matrix(Y_test, preds)

In [ ]:
preds = polysvm.predict(X_train)
plot_confusion_matrix(Y_train, preds)

## Kernel SVM & MNIST

In [ ]:
X, Y = load_mnist_dataset_onevsall(class_id=7)

In [ ]:
Y[Y==0]=-1

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RAND_ST) # 70% training and 30% test

In [ ]:
# normalize mnist data

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
polysvm = svm.SVC(kernel='poly', degree=2)

In [ ]:
polysvm.fit(X_train, Y_train)

In [ ]:
preds = polysvm.predict(X_test)
plot_confusion_matrix(Y_test, preds)

In [ ]:
preds = polysvm.predict(X_train)
plot_confusion_matrix(Y_train, preds)

### Kernel SVM by hand

In [ ]:
# get support vectors
support_idx = ksvm.support_
support_vect = X_train[support_idx] # / 255.0
intercept = ksvm.intercept_
support_lb   = Y_train[support_idx] * np.abs(ksvm.dual_coef_)

In [ ]:
def kernel_rbf(x_i, x_j):
    d = x_i - x_j # this is still a vector
    d2 = np.dot(d, d) # vector squared = number
    return np.exp(-d2 / 2)

In [ ]:
def predict(x_j, support_vect, support_lb):
    
    kv = np.apply_along_axis(lambda x_i: kernel_rbf(x_i, x_j), 1, support_vect)
    preds = kv * support_lb
    return np.sum(preds)


In [ ]:
preds = np.apply_along_axis(lambda x_j : predict(x_j, support_vect, support_lb), 1, X_test[:50] / 255.0)
preds = 2 * (preds > 0) - 1

In [ ]:
plot_confusion_matrix(Y_test[:50], preds)